In [1]:
import os
import openai
import sys
sys.path.append('../..')

openai_api_key  = "Your Key Here"

In [2]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [ ]:
# These are libraries one may need to install if not already done
#! pip install pypdf 
#! pip install chromadb

In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [4]:
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [11]:
!rm -rf ./worldcupData/chromadb  # remove old database files if any
persist_directory = 'worldcupData/chromadb/'

In [12]:
loader = PyPDFDirectoryLoader("worldcupData/1992/")
pages = loader.load()

In [8]:
maxMatchesPlayed = len(pages)
print("The number of world cup matches played in 1992 is "+str(len(pages)))

The number of world cup matches played in 1992 is 39


In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = text_splitter.split_documents(pages)

In [14]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [77]:
question = "How many runs did Ravi Shastri score against England?"
docs = vectordb.max_marginal_relevance_search(question)

In [78]:
smalldb = Chroma.from_documents(
    documents=docs,
    embedding=embedding
)

In [79]:
llm = ChatOpenAI(model_name=llm_name, temperature=0,openai_api_key=openai_api_key)

In [80]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum. Keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [81]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=smalldb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [82]:
result = qa_chain({"query": question})

In [83]:
print(result["result"])

Ravi Shastri scored 57 runs against England. Thanks for asking!
